# Previsão de renda

### 4 elementos importantes
- Esse notebook
- Streamlit com as análises
- Seu Github com o projeto
- Vídeo no readme do github mostrando o streamlit

## Etapa 1 CRISP - DM: Entendimento do negócio

Uma instituição financeira quer conhecer melhor o perfil de renda de seus novos clientes para diversos fins, por exemplo, melhor dimensionar o limite de cartões de crédito dos novos clientes, sem necessariamente solicitar olerites ou documentações que impactem na experiência do seu cliente.

Para isto, conduziu um estudo com alguns clientes, comprovando suas rendas através de olerites e outros documentos, e pretende construir um modelo preditivo para esta renda com base em algumas variáveis que já possui em seu banco de dados.


## Etapa 2 Crisp-DM: Entendimento dos dados



### Dicionário de dados



| Variável                | Descrição                                           | Tipo         |
| ----------------------- |:---------------------------------------------------:| ------------:|
| data_ref                |  Data de referência                                 | Objeto |
| id_cliente              |  Id do cliente                                      | Int |
| sexo                    |  Sexo do cliente                                    | Objeto |
| posse_de_veiculo        |  Possui veiculo?                                    | Booleano |
| posse_de_imovel         |  Possui imovel?                                     | Booleano |
| qtd_filhos              |  Quantos filhos?                                    | Int |
| tipo_renda              |  Tipo de renda                                      | Objeto |
| educacao                |  Escolaridade                                       | Objeto |
| estado_civil            |  Estado civil                                       | Objeto |
| tipo_residencia         |  Tipo de residência                                 | Objeto |
| idade                   |  Idade                                              | Int |
| tempo_emprego           |  Tempo de empregabiilidade                          | Float |
| qt_pessoas_residencia   |  Quantidade de pessoas na residência                | Float |
| renda                   |  Renda                                              | Float |

<div style="text-align: right"


